## 7.14 函数


函数调用可能会降低程序速度，原因如下：

- 函数调用会让微处理器跳转到不同的代码地址然后返回。这可能需要多达4个时钟周期。
在大多数情况下，微处理器能够将调用及返回操作与其他计算重叠运行以节省时间。

- 如果代码碎片化地分散在内存中，代码缓存的效率会降低。

- 函数参数以32位模式存储在堆栈中。
将参数存储在堆栈上并再次读取它们需要额外的时间。
如果某个参数处于关键依赖链条中，因此带来的延时是需要重视的。

- 设置堆栈帧，保存和恢复寄存器，以及保存异常处理信息（可能需要，也可能不需要），都需要额外的时间。

- 每个函数调用语句占用分支目标缓冲区（BTB）中的一个单元。
如果程序的关键部分有很多函数调用和分支，BTB中的争用可能导致分支预测错误

可以使用以下方法，来减少在关键程序的中花费在函数调用上的时间。

### 避免非必要的函数

**一些编程教科书建议每个长于若干行的函数应该分成多个函数。我不赞成此规则。**
将函数拆分为多个较小的函数只会降低程序的效率。
除非函数执行多个逻辑上不同的任务，否则仅仅因为它很长就拆分函数，并不会使程序更加清晰。
可能的话，关键性的最内层环应该完全放入一个函数内。

### 使用内联函数

内联函数可像宏一样扩展，以便调用函数的每个语句都被函数体替换。
如果使用了`inline`关键字或者在类定义中包含了函数体，则通常函数会被内联。
如果函数很小，或者仅从程序中的一个地方调用该函数，则内联函数是有好处的。
小函数通常由编译器自动内联。
另一方面，如果内联导致技术问题或性能问题，编译器在某些情况下可能会忽略内联函数的请求。

### 最内层循环避免函数嵌套

调用其他函数的函数称为**帧函数**，而不调用任何其他函数的函数称为**叶函数**。
叶函数比函数有更高效率，原因如第63页所述。
如果程序的关键部分的最内层循环包含对帧函数的调用，则可通过下面方法来改进代码：
- 内联帧函数
- 把帧函数转换为叶函数，转换方法：内联所有被调用的函数

### 使用宏代替函数

使用`#define`声明的宏肯定会被内联。但请注意，每次使用宏参数时都会对其进行评估。例如：
```cpp
// Example 7.34a. Use macro as inline function
#define MAX(a,b) (a > b ? a : b)
y = MAX(f(x), g(x));
```
在这个例子中, `f(x)` 或 `g(x)` 被计算了两次，原因是它被引用了两次。
你可以通过使用内联函数而不是用宏，来避免这种情况。如果您希望该内联函数支持任何类型的参数，那么将参数设计为模板：
```cpp
// Example 7.34b. Replace macro by template
template <typename T>
static inline T max(T const & a, T const & b) {
 return a > b ? a : b;
}
```

宏的另一个问题是名称不能重载，或者（把名称）限制在作用域中。
无论是在作用域，还是名字空间中，宏都将干扰具有相同名称的任何函数或变量。
因此，为宏使用足够长且唯一的名称非常重要，尤其是在头文件中。

### 使用`fastcall`函数

- 在32位模式下，关键字`__fastcall`更改函数调用方法，以便前两个（CodeGear编译器上是三个）整型参数使用寄存器传输，而不是用堆栈传输。
这可以提高具有整型参数函数的速度。浮点参数不受`__fastcall`的影响。
类的成员函数中的隐式“this”指针也被视为参数，因此可能只剩下一个空闲寄存器来传输其他参数。
因此，在使用`__fastcall`时，请确保最关键的整型参数首先出现在函数参数中。
- 在64位模式下，函数参数默认传输到寄存器中。 因此，在64位模式下无法识别`__fastcall`关键字。


### 函数本地化

仅在同一模块（即当前.cpp文件）中使用的函数应该被设置为本地函数。
这使编译器更容易生成内联函数，并跨函数间进行优化。
有三种方法可以使函数本地化：

1. 将关键字static添加到函数声明中。这是最简单的方法，但它不适用于类成员函数，其中static具有不同的含义。

2. 将函数或类放入匿名的名字空间。

3. Gnu编译器允许使用`"__attribute__((visibility("hidden")))"`.

### 使用整个程序优化

有些编译器有编译选项用于整个程序优化，或者将多个.cpp文件组合到一个目标文件中。
这使编译器能够优化组成程序的所有.cpp模块的寄存器分配和参数传输。
整个程序优化不能用于函数库（以目标文件或库文件的方式分发）。

### 使用64位模式

**参数传输在64位模式下比在32位模式下更高效，在64位Linux中比在64位Windows中更高效。**
- 在64位Linux中，前六个整数参数和前八个浮点参数在寄存器中传输，总计最多十四个寄存器参数。
- 在64位Windows中，前四个参数在寄存器中传输，无论它们是整数还是浮点数。

因此，如果函数具有四个以上的参数，则64位Linux比64位Windows更有效。
在参数传递这方面，32位Linux和32位Windows之间没有区别。
